## Question 3: Nelder-Mead

**Implementation**

In [1]:
import copy
import math
import numpy as np

Function definition

In [2]:
def f1(x):
    return x[0] - x[1] + 2*x[0]**2 + 2*x[0]*x[1]

def f2(x):
    return x[0]**2 + 2*x[1]**2 - 2*x[0]*x[1] - 2*x[1]

In [11]:
def nelder_mead(f, x_start,
                step=0.014, no_improve_thr=10e-3,
                no_improv_break=100, max_iter=0,
                alpha=1., gamma=2., rho=-0.5, sigma=0.5):
    """
        @param f (function): function to optimize, must return a scalar score
            and operate over a numpy array of the same dimensions as x_start
            
        @param x_start (numpy array): initial position
        
        @param step (float): look-around radius in initial step
        
        @no_improv_thr,  no_improv_break (float, int): break after no_improv_break iterations with
            an improvement lower than no_improv_thr
            
        @max_iter (int): always break after this number of iterations.
            Set it to 0 to loop indefinitely.
            
        @alpha, gamma, rho, sigma (floats): parameters of the algorithm
            (see Wikipedia page for reference)
            
        return: tuple (best parameter array, best score)
    """

    # INITIAL SIMPLEX & FUNCTION EVALUATION
    dim = len(x_start)
    prev_best = f(x_start)
    no_improv = 0
    res = [[x_start, prev_best]]

    for i in range(dim):
        x = copy.copy(x_start)
        x[i] = x[i] + step
        score = f(x)
        res.append([x, score])

    # SIMPLEX
    iters = 0
    while 1:
        # order
        res.sort(key=lambda x: x[1])
        best = res[0][1]
        
        # CLOSED BALL RADIUS IMPLEMENTATION
        if np.linalg.norm(x_start-res[0][0]) > 10.:
            break
        
        # BREAK AFTER MAX_ITER
        if max_iter and iters >= max_iter:
            return res[0]
        iters += 1
        
        # BREAK WITH NO IMPROVEMENT
        print 'Iteration:',iters
        print  res[0][0],res[0][1]
        print  res[1][0],res[1][1]
        print  res[2][0],res[2][1]
        print 'Best Energy',best,'\n'
        
        if best < prev_best - no_improve_thr:
            no_improv = 0
            prev_best = best
        else:
            no_improv += 1

        if no_improv >= no_improv_break:
            return res[0]

        # CALCULATE CENTROID
        x0 = [0.] * dim
        for tup in res[:-1]:
            for i, c in enumerate(tup[0]):
                x0[i] += c / (len(res)-1)

        # REFLECTION
        xr = x0 + alpha*(x0 - res[-1][0])
        rscore = f(xr)
        if res[0][1] <= rscore < res[-2][1]:
            del res[-1]
            res.append([xr, rscore])
            continue

        # EXPANSION
        if rscore < res[0][1]:
            xe = x0 + gamma*(x0 - res[-1][0])
            escore = f(xe)
            if escore < rscore:
                del res[-1]
                res.append([xe, escore])
                continue
            else:
                del res[-1]
                res.append([xr, rscore])
                continue

        # CONTRACTION
        xc = x0 + rho*(x0 - res[-1][0])
        cscore = f(xc)
        if cscore < res[-1][1]:
            del res[-1]
            res.append([xc, cscore])
            continue

        # REDUCTION
        x1 = res[0][0]
        nres = []
        for tup in res:
            redx = x1 + sigma*(tup[0] - x1)
            score = f(redx)
            nres.append([redx, score])
        res = nres

\\begin{equation}
(i)\ min \ f(x),\ f(x) = x_1 - x_2 + 2x_1^2 + 2x_1x_2
\\end{equation}

- The algorithm converges in 25 iterations 
-  with α = 1 β = 0.5 γ = 2 , 
-  the iterations begin with the
  triangle points as 
  (1,-1) , (1,-.99) , (1.014 , -1.)

In [15]:
print nelder_mead(f1, np.array([1.,-1.]))

Iteration: 1
[ 1. -1.] 2.0
[ 1.    -0.986] 2.014
[ 1.014 -1.   ] 2.042392
Best Energy 2.0 

Iteration: 2
[ 0.972 -0.979] 1.937392
[ 1. -1.] 2.0
[ 1.    -0.986] 2.014
Best Energy 1.937392 

Iteration: 3
[ 0.958  -0.9965] 1.880734
[ 0.972 -0.979] 1.937392
[ 1. -1.] 2.0
Best Energy 1.880734 

Iteration: 4
[ 0.895   -0.96325] 1.7360825
[ 0.958  -0.9965] 1.880734
[ 0.972 -0.979] 1.937392
Best Energy 1.7360825 

Iteration: 5
[ 0.8355   -0.981625] 1.572950125
[ 0.895   -0.96325] 1.7360825
[ 0.958  -0.9965] 1.880734
Best Energy 1.572950125 

Iteration: 6
[ 0.67975   -0.9243125] 1.27157978125
[ 0.8355   -0.981625] 1.572950125
[ 0.895   -0.96325] 1.7360825
Best Energy 1.27157978125 

Iteration: 7
[ 0.482875   -0.93240625] 0.981146445312
[ 0.67975   -0.9243125] 1.27157978125
[ 0.8355   -0.981625] 1.572950125
Best Energy 0.981146445312 

Iteration: 8
[ 0.0729375  -0.82182813] 0.785521205078
[ 0.482875   -0.93240625] 0.981146445312
[ 0.67975   -0.9243125] 1.27157978125
Best Energy 0.785521205078 



\\begin{equation}
(ii)\ min \ f(x),\ f(x) = x_1^2 + 2x_2^2 - 2x_1x_2 - 2x_2
\\end{equation}

- The algorithm converges in 128 iterations 
-  with α = 1 β = 0.5 γ = 2 , the iterations begin with the
-  triangle points as (1.,-0.986) , (1.,-1.) , (1.014,-1.)

In [16]:
print nelder_mead(f2, np.array([1.,-1.]))

Iteration: 1
[ 1.    -0.986] 6.888392
[ 1. -1.] 7.0
[ 1.014 -1.   ] 7.056196
Best Energy 6.888392 

Iteration: 2
[ 0.972 -0.979] 6.722842
[ 1.    -0.986] 6.888392
[ 1. -1.] 7.0
Best Energy 6.722842 

Iteration: 3
[ 0.958  -0.9475] 6.4236865
[ 0.972 -0.979] 6.722842
[ 1.    -0.986] 6.888392
Best Energy 6.4236865 

Iteration: 4
[ 0.895   -0.91775] 5.963827625
[ 0.958  -0.9475] 6.4236865
[ 0.972 -0.979] 6.722842
Best Energy 5.963827625 

Iteration: 5
[ 0.8355   -0.839875] 5.19202140625
[ 0.895   -0.91775] 5.963827625
[ 0.958  -0.9475] 6.4236865
Best Energy 5.19202140625 

Iteration: 6
[ 0.67975   -0.7414375] 4.05237847656
[ 0.8355   -0.839875] 5.19202140625
[ 0.895   -0.91775] 5.963827625
Best Energy 4.05237847656 

Iteration: 7
[ 0.482875   -0.53646875] 2.39979790039
[ 0.67975   -0.7414375] 4.05237847656
[ 0.8355   -0.839875] 5.19202140625
Best Energy 2.39979790039 

Iteration: 8
[ 0.0729375  -0.23710937] 0.62656867041
[ 0.482875   -0.53646875] 2.39979790039
[ 0.67975   -0.7414375] 4.052